# Simulating lightcurves for SNe Ia

This notebooks shows an example of a simulating SN Ia lightcurves based on the SALT2 template for 90 days of ZTF observations. (The survey is restricted to 90 days in order to run reasonably fast. A similar simulation for a 3-year survey can be generated using SNIa_3yr.py.)

In [17]:
import warnings
## No annoying warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import simsurvey.cadence as simul

import simsurvey_tools as sst

In [18]:
# Set the time frame for the survey (90 days)
mjd_range = (58000, 58090)

The transient generator randomly draws all parameters needed to simulate the lightcurves. For the SALT2 template most things are predefined. The only input required is the frame, which should be set to include a number of days before and after the survey in order to simulate all SNe that may be visible during the observations. Additionally, to reduce the computation time, the  coordinate ranges can be restricted such that only few SNe outside the observed part of the sky are simulated.

In [7]:
tr = simul.get_sn_generator([0.,0.2], ratekind="basic", 
                            dec_range=[-40,90],
                            mjd_range=[mjd_range[0] - 60, mjd_range[-1] + 24])

The plan object collects all required information of the survey strategy. The function used below generates a simple mock survey to get a basic idea of what we can expect from ZTF, see its docstring in `simsurvey_tools.py` for more information. The output of Eric's `ztf_sim` code can also be loaded into a plan object, see `plan_from_ztf_sim.ipynb`. (Currently the code uses DES $gri$ filters instead of the ZTF ones, which be included later.)

In [10]:
plan = sst.get_survey_plan_simple(mjd_range=mjd_range)

In [13]:
plan.cadence

band,skynoise,field,RA,time,Dec
str4,float64,int64,float64,float64,float64
desr,800.0,661,57.69565,58000.0,31.75385
desr,800.0,666,96.82609,58000.0005208,31.75385
desr,800.0,662,65.52174,58000.0010417,31.75385
desr,800.0,659,42.04348,58000.0015625,31.75385
desr,800.0,658,34.21739,58000.0020833,31.75385
desr,800.0,667,104.65217,58000.0026042,31.75385
desr,800.0,665,89.0,58000.003125,31.75385
desr,800.0,654,2.91304,58000.0036458,31.75385
desr,800.0,656,18.56522,58000.0041667,31.75385


Lastly, some basic properties of the instrument used in the survey must be defined but these are mostly chosen in order to get specific depths of the observations and need not be changed.

In [14]:
instprop = {"desg":{"gain":1.,"zp":30,"zpsys":'ab'},
            "desr":{"gain":1.,"zp":30,"zpsys":'ab'},
            "desi":{"gain":1.,"zp":30,"zpsys":'ab'}}

These three components are collected in a SimulSurvey object, which is then used to generate the lightcurves.

In [15]:
survey = simul.SimulSurvey(generator=tr, 
                           plan=plan, 
                           instprop=instprop)

lcs = survey.get_lightcurves(
    # This cell may give an error because of the progress bar widget.
    # Just delete the following line to fix this.
    progress_bar=True, notebook=True
)

Determining field IDs for all objects

Generating lightcurves



The output is a LightcurveCollection object that organizes the lightcurves and can be accessed as shown below.

In [16]:
lcs[0]

time,band,flux,fluxerr,zp,zpsys
float64,str4,float64,float64,int64,str2
58000.0817708,desr,976.294141652,800.671506018,30,ab
58000.1651042,desr,1016.71805025,800.669063892,30,ab
58005.0822917,desr,407.243481752,800.53302942,30,ab
58005.165625,desr,176.673854044,800.530805205,30,ab
58006.08125,desr,648.443789384,800.506535696,30,ab
58006.1645833,desr,950.798403662,800.504342782,30,ab
58011.0817708,desr,1473.66450853,800.388890557,30,ab
58011.1651042,desr,405.637855062,800.387382669,30,ab
58019.0807292,desr,1113.23450336,800.298638615,30,ab
